In [4]:
import os

from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.messages import HumanMessage

from langgraph.checkpoint.memory import MemorySaver

from langgraph.prebuilt import create_react_agent

In [ ]:
os.environ['OPENAI_API_KEY'] = ''
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["TAVILY_API_KEY"] = ''

In [6]:
search = TavilySearchResults(max_results=2)

In [15]:
search_results = search.invoke("Which country won the Euro 2024?")

In [16]:
search_results

[{'url': 'https://www.cnn.com/2024/07/14/sport/spain-england-euro-2024-final-spt-intl/index.html',
  'content': "Spain won a record-breaking fourth European Championship, defeating England 2-1 following a drama-filled second half in the Euro 2024 final on Sunday in Berlin.. Nico Williams - one of Spain's ..."},
 {'url': 'https://www.bbc.com/sport/football/articles/cv2gvreml2ko',
  'content': "Football often divides opinion - but everyone was united to hail Spain's Euro 2024 success as fully deserved. La Roja won all seven games without needing penalties, a record at a European ..."}]

In [17]:
tools = [search]

In [19]:
model = ChatOpenAI(model="gpt-4o-mini")

In [20]:
model_with_tools = model.bind_tools(tools)

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [31]:
response.dict()

{'content': 'Hello! How can I assist you today?',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 10,
   'prompt_tokens': 81,
   'total_tokens': 91},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_f33667828e',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-f7028967-453a-49dc-9f1a-029d0474a88a-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 81,
  'output_tokens': 10,
  'total_tokens': 91}}

In [32]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Tokyo?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_c4z7cShpPkfTmykBrUAh6GZ4', 'type': 'tool_call'}]


In [33]:
response.dict()

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_c4z7cShpPkfTmykBrUAh6GZ4',
    'function': {'arguments': '{"query":"current weather in Tokyo"}',
     'name': 'tavily_search_results_json'},
    'type': 'function'}],
  'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 21,
   'prompt_tokens': 85,
   'total_tokens': 106},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_9722793223',
  'finish_reason': 'tool_calls',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-8b96e22a-ea41-4acb-84c4-0b0361857afe-0',
 'example': False,
 'tool_calls': [{'name': 'tavily_search_results_json',
   'args': {'query': 'current weather in Tokyo'},
   'id': 'call_c4z7cShpPkfTmykBrUAh6GZ4',
   'type': 'tool_call'}],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 85,
  'output_tokens': 21,
  'total_tokens': 106}}

In [34]:
agent_executor = create_react_agent(model, tools)

In [35]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

In [38]:
response['messages'][1].dict()

{'content': 'Hello! How can I assist you today?',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 10,
   'prompt_tokens': 81,
   'total_tokens': 91},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_f33667828e',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-a08f4ed5-8d2e-40f0-bfff-e461456794de-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 81,
  'output_tokens': 10,
  'total_tokens': 91}}

In [39]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Tokyo?")]}
)

In [42]:
for msg in response['messages']:
    print(f'- {msg.content}')

- whats the weather in Tokyo?
- 
- [{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.69, 'lon': 139.69, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1724993389, 'localtime': '2024-08-30 13:49'}, 'current': {'last_updated_epoch': 1724993100, 'last_updated': '2024-08-30 13:45', 'temp_c': 26.0, 'temp_f': 78.8, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 170, 'wind_dir': 'S', 'pressure_mb': 1007.0, 'pressure_in': 29.74, 'precip_mm': 2.47, 'precip_in': 0.1, 'humidity': 100, 'cloud': 75, 'feelslike_c': 29.3, 'feelslike_f': 84.7, 'windchill_c': 26.6, 'windchill_f': 79.9, 'heatindex_c': 30.4, 'heatindex_f': 86.7, 'dewpoint_c': 24.4, 'dewpoint_f': 75.8, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 6.0, 'gust_mph': 25.7, 'gust_kph': 41.3}}"}, {"url": "https://www.meteoprog.com/weather/Tokyo/month/

In [43]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Tokyo?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1w830HAIZwM4K2ZHLkFZuAw6', 'function': {'arguments': '{"query":"current weather in Tokyo"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_5bd87c427a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f34d3802-6131-4793-9cfe-1ff1ea0b5424-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_1w830HAIZwM4K2ZHLkFZuAw6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 21, 'total_tokens': 107})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Tokyo\', \'region\': \'Tokyo\', \'country\': \'Japan\', \'lat\': 35.69, \'lon\': 1

In [44]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Tokyo?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/tmp/ipykernel_30638/2392252406.py:1: LangChainBetaWarning: This API is in beta and may change in the future.
  async for event in agent_executor.astream_events(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in Tokyo'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Tokyo\', \'region\': \'Tokyo\', \'country\': \'Japan\', \'lat\': 35.69, \'lon\': 139.69, \'tz_id\': \'Asia/Tokyo\', \'localtime_epoch\': 1724993597, \'localtime\': \'2024-08-30 13:53\'}, \'current\': {\'last_updated_epoch\': 1724993100, \'last_updated\': \'2024-08-30 13:45\', \'temp_c\': 26.0, \'temp_f\': 78.8, \'is_day\': 1, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/296.png\', \'code\': 1183}, \'wind_mph\': 11.9, \'wind_kph\': 19.1, \'wind_degree\': 170, \'wind_dir\': \'S\', \'pressure_mb\': 1007.0, \'pressure_in\': 29.74, \'precip_mm\': 2.47, \'precip_in\': 0.1, \'humidity\': 100, \'cloud\': 75, \'feelslike_c\': 29.3, \'feelslike_f\': 84.7, \'windchill_c\': 26.6, \'windchill_f\': 79.9, \'heatindex_

In [45]:
memory = MemorySaver()

In [46]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [49]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Jose!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello again, Jose! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 113, 'total_tokens': 126}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-032d0aec-1a41-4180-b888-68d64532724d-0', usage_metadata={'input_tokens': 113, 'output_tokens': 13, 'total_tokens': 126})]}}
----


In [50]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Jose!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 138, 'total_tokens': 144}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a2178c5-45a5-439e-84c9-81f00694ad09-0', usage_metadata={'input_tokens': 138, 'output_tokens': 6, 'total_tokens': 144})]}}
----
